# Importing all required libraries

In [1]:
import os
import requests
import json
cwd = os.getcwd()

In [453]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
import seaborn as sb
import numpy as np
import pandas as pd
from pandas import Series
import zipfile
from bs4 import BeautifulSoup
import re

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
rcParams['figure.figsize'] = 5, 4
sb.set_style('whitegrid')


pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# Gathering Data

## Getting the Twitter Archive

In [507]:
df = pd.read_csv('twitter-archive-enhanced.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Getting the extra data from Twitter API

In [ ]:
import tweepy

consumer_key = 'ejjJsGZP91PETUTLFQnl1I47g'
consumer_secret = 'tT2dnqoWcJonM1RGCVkMl6net7kBYRsUsHnD12r5MpAI35Zdwv'
access_token = '49008389-54oYNUfoyb0GGuaKRSWlhZQKn19RQq9AI5ZwqaRdA'
access_secret = 'K4qvZSLINbo8tnF7e3AJtEQR8tn3GVHuIX3joiUfptrbV'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

### Writing a JSON file storing all the tweets

In [ ]:
tweet = []
tweet_dict = {"tweet_id" :[], "tweet_status" :[],
              "full_text" :[], "retweet_count": [], "favorite_count" : []}
for ids in enumerate(df.tweet_id):
    try:
        status = api.get_status(ids[1], tweet_mode='extended')
        
        tweet.append(status)
        tweet_dict["tweet_status"].append(status)
        tweet_dict["tweet_id"].append(status.id)
        tweet_dict["full_text"].append(status.full_text)
        tweet_dict["retweet_count"].append(status.retweet_count)
        tweet_dict["favorite_count"].append(status.favorite_count)
    except:
        pass

### converting the created doctionary into a Pandas DataFrame

In [ ]:
tweet_df = pd.DataFrame.from_dict(tweet_dict );

#### Storing the Data Frame into a CSV file

In [ ]:
tweet_df.to_csv('tweet_df.csv')

### Writing the Tweet Statuses into a txt file:

In [ ]:
pd.DataFrame(tweet).to_csv(r'tweet_json.txt', header=None, index=None, sep=' ', mode='a')

### Reading the JSON data from the "tweet_json.txt" into a DataFrame

In [ ]:
json_df = pd.read_csv('tweet_json.txt', header = None)

## Downloading the TSV file Programmatically

In [ ]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

response = requests.get(url)
with open("image_predictions.tsv", mode='wb') as file:
        file.write(response.content)

### Reading the TSV file into a DataFrame

In [511]:
df_immage = pd.read_csv('image_predictions.tsv', sep = '\t')
df_immage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


# Assessing Data

## Assessing the Twitter Archive

In [3]:
df = pd.read_csv('twitter-archive-enhanced.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## <font color='red'>Issues within the Tweeter Archive DataFrame:</font>

`Tidyness Issues`
- T-1 ** denominator of 10 and the numerators could all be in one column**
- T-2 ** dog stages need to be in one single column (doggo, floofer, pupper, puppo)**
- T-3 ** it's not clear if the tweet is the original satus or a retweet of something else, so we need to add a column for Retweet (Yes/No)**


`Quality Issues`
- Q-1 ** too many NaNs in (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp)**
- Q-2 **source column is not clear and we need to extract the exact source like iPhone, etc.**
- Q-3 **for Dos Stage and Name, for missing values, "None" is used rather than NaN**
- Q-4 **timestamp is not standard, so we could change it to a standard format of Date-time**
- Q-7 ** after replacing None with NaN, we see that a lot of Dog Stages are missing!**
- Q-9 ** All other ID fileds are also need to be converted into String like `in_reply_to_status_id`**


## Assessing the Image Prediction DataFrame

In [8]:
df_image = pd.read_csv('image_predictions.tsv', sep = '\t')
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
df_image.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
319,671768281401958400,https://pbs.twimg.com/media/CVKZsHtWwAA6gPj.jpg,2,Chihuahua,0.500373,True,French_bulldog,0.112796,True,Italian_greyhound,0.062893,True
1695,816336735214911488,https://pbs.twimg.com/media/C1Q17WdWEAAjKFO.jpg,1,Labrador_retriever,0.919330,True,kuvasz,0.049480,True,golden_retriever,0.011934,True
350,672488522314567680,https://pbs.twimg.com/media/CVUovvHWwAAD-nu.jpg,1,Doberman,0.605358,True,Rottweiler,0.108382,True,Appenzeller,0.077798,True
982,707387676719185920,https://pbs.twimg.com/media/CdElVm7XEAADP6o.jpg,1,Chihuahua,0.888468,True,Italian_greyhound,0.088635,True,toy_terrier,0.015938,True
903,700062718104104960,https://pbs.twimg.com/media/CbcfUxoUAAAlHGK.jpg,1,hummingbird,0.180998,False,peacock,0.135179,False,eel,0.075371,False
1618,802572683846291456,https://pbs.twimg.com/media/CyNPmJgXcAECPuB.jpg,1,golden_retriever,0.610171,True,Labrador_retriever,0.173252,True,cocker_spaniel,0.163257,True
65,667174963120574464,https://pbs.twimg.com/media/CUJIFoJWsAAL3Dc.jpg,1,toy_poodle,0.266437,True,Chihuahua,0.243223,True,bluetick,0.072806,True
134,668484198282485761,https://pbs.twimg.com/media/CUbu1GAWsAEH3E-.jpg,1,standard_poodle,0.587372,True,Bedlington_terrier,0.182411,True,Afghan_hound,0.040968,True
823,693155686491000832,https://pbs.twimg.com/media/CZ6VatdWwAAwHly.jpg,3,Shih-Tzu,0.697480,True,Lhasa,0.200151,True,Tibetan_terrier,0.090970,True
758,688789766343622656,https://pbs.twimg.com/media/CY8SocAWsAARuyh.jpg,1,American_Staffordshire_terrier,0.599660,True,Staffordshire_bullterrier,0.380976,True,bull_mastiff,0.003889,True


In [420]:
df_image.p1.value_counts();

## <font color='red'>Issues within the Image Prediction DataFrame:</font>

`Tidyness Issues:`
- T-4 **The Columns titles are not descriptive enough (img_num, p(1-3), p(1-3)_dog)**
- T-5 **number of records for this DatFrame is not the same as the archive. It has 2075 records while he other has 2356 records**

`quality Issues:`
- Q-5 **Some of the photos are not dog (like the p1 says: hummingbird, p2: peacock, p3: eel)**
- Q-6 **some of the names in the p1-p3 are not real dog breads (web-site, llama, tub, etc.)**





## Assessing the Tweet extra data - DataFrame

In [34]:
df_tweet = pd.read_csv('tweet_df.csv')
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 6 columns):
Unnamed: 0        2345 non-null int64
favorite_count    2345 non-null int64
full_text         2345 non-null object
retweet_count     2345 non-null int64
tweet_id          2345 non-null int64
tweet_status      2345 non-null object
dtypes: int64(4), object(2)
memory usage: 110.0+ KB


In [35]:
df_tweet.head()

,Unnamed: 0,favorite_count,full_text,retweet_count,tweet_id,tweet_status
0,0,38994,This is Phineas. He's a mystical boy. Only eve...,8653,892420643555336193,"Status(truncated=False, _api=<tweepy.api.API o..."
1,1,33390,This is Tilly. She's just checking pup on you....,6355,892177421306343426,"Status(truncated=False, _api=<tweepy.api.API o..."
2,2,25168,This is Archie. He is a rare Norwegian Pouncin...,4220,891815181378084864,"Status(truncated=False, _api=<tweepy.api.API o..."
3,3,42347,This is Darla. She commenced a snooze mid meal...,8768,891689557279858688,"Status(truncated=False, _api=<tweepy.api.API o..."
4,4,40514,This is Franklin. He would like you to stop ca...,9531,891327558926688256,"Status(truncated=False, _api=<tweepy.api.API o..."


In [46]:
df_tweet.full_text[1]

"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV"

## <font color='red'>Issues within the Tweet DataFrame:</font>

`Tidyness Issues:`
- T-6 **Columns are not optimally arranged**
- T-7 **no need for Unnamed id column at the begining**
- T-8 **Should merge 2 DataFrames (`Tweet_DataFrame` and `Tweeter_archive_enhaced`)**

    
`Quality Issues:`
- Q-7 **full text contain some url and also rating which are irrelevant, need to be removed**
- Q-8 **Tweet_id is in Integer form, needs to be string**

# Clean

First we are going to create copies of the data Frames to work

In [55]:
df_clean = df.copy()

In [48]:
df_tweet_clean = df_tweet.copy()

In [468]:
df_image_clean = df_image.copy()

## Issue `Q3`

### Define:
We are going to replace `"None"` with NaN or Null (a missing value).

will be working with __`df_clean`__

## code:

In [61]:
df_clean = df_clean.replace('None',np.NaN)

## Test

In [62]:
df_clean.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1621,684926975086034944,NaN,NaN,2016-01-07 02:38:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Bruiser &amp; Charlie. They are the best ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/684926975...,11,10,Bruiser,NaN,NaN,NaN,NaN


In [101]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
doggo                         97 non-null object
floofer                       10 non-null object
pupper                        257 non-null object
puppo                         30 non-null object
dtypes: float64(4), int64(3), object(10

## Issue `Q7`

### Define

Now we have replaced the None with NaNs and we see that a lot of `Dog Stages` are missing.
We are going to read the Tweet texts and see if we can find a mention of the dog stage.

will be working with __`df_clean`__

In [110]:
len(df_clean[df_clean['text'].str.contains("doggo|puppo|floof|pupper")])

418

As we can see from the code above, there are 418 mentions of the words [doggo, puppo, floof, pupper]. So we need to:

- first change the title of floofer to floof(er)
- find the ones that are missing in the df file (418 - 394 = 24) are still left without a dog stage

### Code

create a list of all records in the `Original Data Farme` that have at least one of the `doggo,puppo,floof,pupper` titles in the fileds

In [411]:
df_clean[df_clean['text'].str.contains("doggo|puppo|floof|pupper")];

In [154]:
df_clean['dog_stage'] = df_clean['text'].str.extract('(doggo|puppo|floof|pupper)',expand=True)

In [412]:
df_clean[df_clean['dog_stage'].notnull()];

**So we drop the 4 dog stage columns and replace it with one that we just created by extracting the `Dog Stage` from the text**

In [171]:
df_clean = df_clean.drop(["doggo", "floofer", "pupper", "puppo"], axis=1)

### Issue `Q-9`

The issue with all ID fields, they all need to be converted into `string`

### Define

We are going to change the type of the ID columns into String. These columns include:

- tweet_id
- in_reply_to_status_id
- in_reply_to_user_id
- retweeted_status_id
- retweeted_status_user_id

will be working with __`df_clean`

### Code

In [197]:
df_clean['tweet_id'] = df_clean['tweet_id'].fillna(0.0)
df_clean['in_reply_to_status_id'] = df_clean['in_reply_to_status_id'].fillna(0.0)
df_clean['in_reply_to_user_id'] = df_clean['in_reply_to_user_id'].fillna(0.0)
df_clean['retweeted_status_id'] = df_clean['retweeted_status_id'].fillna(0.0)
df_clean['retweeted_status_user_id'] = df_clean['retweeted_status_user_id'].fillna(0.0)

In [198]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null float64
in_reply_to_user_id           2356 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null float64
retweeted_status_user_id      2356 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
dog_stage                     418 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 257.8+ KB


In [199]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype('int64')
df_clean['in_reply_to_status_id'] = df_clean['in_reply_to_status_id'].astype('int64')
df_clean['in_reply_to_user_id'] = df_clean['in_reply_to_user_id'].astype('int64')
df_clean['retweeted_status_id'] = df_clean['retweeted_status_id'].astype('int64')
df_clean['retweeted_status_user_id'] = df_clean['retweeted_status_user_id'].astype('int64')

In [200]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
dog_stage                     418 non-null object
dtypes: int64(7), object(7)
memory usage: 257.8+ KB


In [202]:
df_clean['tweet_id'] = df_clean['tweet_id'].astype('str')
df_clean['in_reply_to_status_id'] = df_clean['in_reply_to_status_id'].astype('str')
df_clean['in_reply_to_user_id'] = df_clean['in_reply_to_user_id'].astype('str')
df_clean['retweeted_status_id'] = df_clean['retweeted_status_id'].astype('str')
df_clean['retweeted_status_user_id'] = df_clean['retweeted_status_user_id'].astype('str')

Replacing the remaining 0s with NaN

In [211]:
df_clean['tweet_id'] = df_clean['tweet_id'].replace('0', np.NaN)
df_clean['in_reply_to_status_id'] = df_clean['in_reply_to_status_id'].replace('0', np.NaN)
df_clean['in_reply_to_user_id'] = df_clean['in_reply_to_user_id'].replace('0', np.NaN)
df_clean['retweeted_status_id'] = df_clean['retweeted_status_id'].replace('0', np.NaN)
df_clean['retweeted_status_user_id'] = df_clean['retweeted_status_user_id'].replace('0', np.NaN)

### test

In [212]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null object
in_reply_to_user_id           78 non-null object
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null object
retweeted_status_user_id      181 non-null object
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
dog_stage                     418 non-null object
dtypes: int64(2), object(12)
memory usage: 257.8+ KB


In [213]:
df_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN


## Issue `T3`

The issue with not knowing which one is `Retweet` which ones are original status!

### Define

We are going to make a column titled **`Retweet`** and then assign True and False to it

will be working with __`df_clean_1`__

In [415]:
df_clean_1 = df_clean.copy()

In [416]:
df_clean_1.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'dog_stage'], dtype='object')

In [417]:
df_clean_1[df_clean_1['retweeted_status_user_id'].notnull()];

### code

In [345]:
index = df_clean_1['retweeted_status_id'].index[df_clean_1['retweeted_status_id'].notnull()]
df_clean_1 = df_clean_1.drop(index)

In [346]:
df_clean_1['retweeted_status_id'].value_counts()

Series([], Name: retweeted_status_id, dtype: int64)

 <font color='red'>So, as we see above, we have `2175 records` left after we remove the retweets. What we are left with  are all `original statuses`.</font>

__So we drop the 3 columns related to `Retweets`. These columns are:__
- retweeted_status_id
- retweeted_status_user_id
- retweeted_status_timestamp

In [347]:
df_clean_1 = df_clean_1.drop(columns=['retweeted_status_id', 'retweeted_status_user_id',
                                     'retweeted_status_timestamp'])

### Test

In [348]:
df_clean_1.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
2191,668955713004314625,NaN,NaN,2015-11-24 00:54:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Slovakian Helter Skelter Feta named ...,https://twitter.com/dog_rates/status/668955713...,10,10,a,NaN


## Issue `T-1`

The issue with the `denominators and numerators`.

will be working with `df_clean_1`

### Define

We need to create one _`single rate`_ and put it in **`one single column`**.

### code

In [349]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
dog_stage                383 non-null object
dtypes: int64(2), object(9)
memory usage: 203.9+ KB


In [350]:
df_clean_1['rating'] = (df_clean_1['rating_numerator'] / df_clean_1['rating_denominator'])

In [351]:
df_clean_1 = df_clean_1.drop(columns=['rating_numerator', 'rating_denominator'])

### test

In [352]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
name                     1495 non-null object
dog_stage                383 non-null object
rating                   2175 non-null float64
dtypes: float64(1), object(9)
memory usage: 186.9+ KB


## Issue `Q-2`

**`source column` is not clear and we need to extract the exact source like iPhone, etc.**

### Define

we need to evaluate how many different sources there are and how we could extract them from the source column.

will be working with `df_clean_1`

### code

In [353]:
df_clean_1['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2042
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       31
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

So, as we can see above there are only `4 different sources`:

- `Twitter for iPhone`
- `Vine`
- `Twitter Web Client`
- `TweetDeck`

In [354]:
df_clean_1['source'] = df_clean_1['source'].str.extract('(iPhone|Vine|Web|tweetdeck)',expand=True)

### test

In [355]:
df_clean_1['source'].value_counts()

iPhone       2042
Vine           91
Web            31
tweetdeck      11
Name: source, dtype: int64

In [356]:
df_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
name                     1495 non-null object
dog_stage                383 non-null object
rating                   2175 non-null float64
dtypes: float64(1), object(9)
memory usage: 186.9+ KB


## Issue `Q-4`

**`timestamp`** is not standard, so we could change it to a standard format of Date-time

### Define

Convert `timestamp` into a correctly formatted date-time column

will be working with `df_clean_2`

### code

First we need to strip the `"+0000"` from the values

In [370]:
df_clean_2 = df_clean_1.copy()

In [371]:
df_clean_2['timestamp'] = df_clean_2['timestamp'].str.strip(" +0000")

In [372]:
df_clean_2['timestamp'].sample()

1031    2016-06-22 01:06:43
Name: timestamp, dtype: object

<font color='red'>**We need to remove the `Second` part of the time, as we have multiple different formats. The best way to deal with it, is to remove the second portion of the timestamp.**</font>

Finding the index of 3 different lengthes!

- one is full format='%Y-%m-%d %H:%M:%S'(`19 characters`)
- second if format='%Y-%m-%d %H:%M:%' (`18 characters`)
- minute if format='%Y-%m-%d %H:%M:' (`17 characters`)

In [373]:
index_full = df_clean_2['timestamp'].index[df_clean_2['timestamp'].str.len()==19]
index_second = df_clean_2['timestamp'].index[df_clean_2['timestamp'].str.len()==18]
index_minute = df_clean_2['timestamp'].index[df_clean_2['timestamp'].str.len()==17]

In [374]:
df_clean_2['timestamp'][index_full] = df_clean_2['timestamp'][index_full].str[:-3]
df_clean_2['timestamp'][index_second] = df_clean_2['timestamp'][index_second].str[:-2]
df_clean_2['timestamp'][index_minute] = df_clean_2['timestamp'][index_minute].str[:-1]

C:\Users\Rahman\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Rahman\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Rahman\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [377]:
df_clean_2['timestamp'].sample()

1663    2016-01-01 06:22
Name: timestamp, dtype: object

Now, we convert it into a Date/Time Format using _`pd.to_datetime`_ function

In [378]:
df_clean_2['timestamp'] = pd.to_datetime(df_clean_2['timestamp'], format='%Y-%m-%d %H:%M')

### test

In [382]:
type(df_clean_2['timestamp'][0])

pandas._libs.tslib.Timestamp

## Issues `Q-8`, `T-6`, and `T-7`

- **`tweet_id` is not string and we need to convert it**
- **`unnammed` column needs to be removed**
- **`arrange` the columns in a better way**

### Define

Dealing with the issues listed above, we will be rearranging the columns, removing a column, and changing the tweet_id type from numeric to string!

will be working with `df_tweet_clean`

### code

In [397]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 6 columns):
Unnamed: 0        2345 non-null int64
favorite_count    2345 non-null int64
full_text         2345 non-null object
retweet_count     2345 non-null int64
tweet_id          2345 non-null int64
tweet_status      2345 non-null object
dtypes: int64(4), object(2)
memory usage: 110.0+ KB


<font color='red'>**Drop the unnecessary column**</font>

In [400]:
df_tweet_clean = df_tweet_clean.drop(df_tweet_clean.columns[0], axis=1)

In [403]:
df_tweet_clean.sample()

,favorite_count,full_text,retweet_count,tweet_id,tweet_status
783,0,RT @dog_rates: Meet Fizz. She thinks love is a...,5760,773336787167145985,"Status(truncated=False, _api=<tweepy.api.API o..."


<font color='red'>**Now rearrange the columns:**</font>

In [406]:
cols = ['tweet_id', 'tweet_status', 'full_text', 'retweet_count', 'favorite_count']

In [418]:
df_tweet_clean = df_tweet_clean[cols]

In [419]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 5 columns):
tweet_id          2345 non-null object
tweet_status      2345 non-null object
full_text         2345 non-null object
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(2), object(3)
memory usage: 91.7+ KB


<font color='red'>**convert tweet_id into string**</font>

In [409]:
df_tweet_clean['tweet_id'] = df_tweet_clean['tweet_id'].astype('str')

### Test

In [410]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 5 columns):
tweet_id          2345 non-null object
tweet_status      2345 non-null object
full_text         2345 non-null object
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(2), object(3)
memory usage: 91.7+ KB


## Issues `T-8`

**Should merge 2 DataFrames (`Tweet_DataFrame` and `Tweeter_archive_enhaced`)**

### Define

We need to have 2 data frames in total, one for the image predictions and one for tweet info. At the moment , we have 2 seperate data frames for tweets, one was imported from the tweeter_archive_enhanced CSV file and one that we extracted from Twitter directly.
So, here I am going to `merge` these 2 dataframes into one complete data frame for tweets.

will be working with `df_tweet_clean` and `df_clean_2`.

### code

In [421]:
df_tweet_clean.sample()

,tweet_id,tweet_status,full_text,retweet_count,favorite_count
682,787111942498508800,"Status(truncated=False, _api=<tweepy.api.API o...",RT @dog_rates: This is Carl. He's very powerfu...,7607,0


In [423]:
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 5 columns):
tweet_id          2345 non-null object
tweet_status      2345 non-null object
full_text         2345 non-null object
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(2), object(3)
memory usage: 91.7+ KB


In [422]:
df_clean_2.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,dog_stage,rating
679,789137962068021249,NaN,NaN,2016-10-20 16:15:00,iPhone,This is Bo. He's a West Congolese Bugaboop Snu...,https://twitter.com/dog_rates/status/789137962...,Bo,NaN,1.2


In [424]:
df_clean_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id                 2175 non-null object
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
name                     1495 non-null object
dog_stage                383 non-null object
rating                   2175 non-null float64
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 266.9+ KB


**We are going to merge the 2 data frames on the `tweet_id`.**

In [429]:
tweet_cleaned = pd.merge(df_clean_2, df_tweet_clean, on='tweet_id')

In [506]:
tweet_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2174
Data columns (total 13 columns):
tweet_id                 2175 non-null object
tweet_status             2175 non-null object
full_text                2175 non-null object
timestamp                2175 non-null datetime64[ns]
source                   2175 non-null object
expanded_urls            2117 non-null object
name                     1495 non-null object
dog_stage                383 non-null object
rating                   2175 non-null float64
retweet_count            2175 non-null int64
favorite_count           2175 non-null int64
in_reply_to_status_id    78 non-null object
in_reply_to_user_id      78 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 237.9+ KB


now, we notice that we have `full_text` in one dataframe and `text` in the other. So we need to drop one of them.

In [431]:
tweet_cleaned = tweet_cleaned.drop(tweet_cleaned.columns['text'], axis=1)

**I am going to rearrange the columns in a better form**

In [434]:
# column names:
list(tweet_cleaned)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'expanded_urls',
 'name',
 'dog_stage',
 'rating',
 'tweet_status',
 'full_text',
 'retweet_count',
 'favorite_count']

In [436]:
cols = ['tweet_id', 'tweet_status', 'full_text', 'timestamp', 'source', 'expanded_urls',
       'name', 'dog_stage', 'rating', 'retweet_count', 'favorite_count', 'in_reply_to_status_id',
        'in_reply_to_user_id']
cols

['tweet_id',
 'tweet_status',
 'full_text',
 'timestamp',
 'source',
 'expanded_urls',
 'name',
 'dog_stage',
 'rating',
 'retweet_count',
 'favorite_count',
 'in_reply_to_status_id',
 'in_reply_to_user_id']

In [439]:
tweet_cleaned = tweet_cleaned[cols]

### Test

In [440]:
tweet_cleaned.sample()

,tweet_id,tweet_status,full_text,timestamp,source,expanded_urls,name,dog_stage,rating,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id
1415,686377065986265092,"Status(truncated=False, _api=<tweepy.api.API o...",Meet Birf. He thinks he's gone blind. 10/10 ve...,2016-01-11 02:40:00,iPhone,https://twitter.com/dog_rates/status/686377065...,Birf,pupper,1.0,626,2385,NaN,NaN


In [443]:
len(tweet_cleaned)

2175

Storing the new DataFrame into a CSV file called `tweet_cleaned.csv`.

In [444]:
tweet_cleaned.to_csv('tweet_cleaned.csv')

## Issues `T-4`

### Define

Image Prediction Data Frame has some columns with rather vague and unclear names. So I am going to change them into something more descriptive.

will be working with `df_image_clean`

In [469]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [470]:
df_image_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1370,762035686371364864,https://pbs.twimg.com/ext_tw_video_thumb/76203...,1,home_theater,0.063152,False,cash_machine,0.046692,False,theater_curtain,0.046277,False
729,686606069955735556,https://pbs.twimg.com/media/CYdQktMWsAEI29_.jpg,1,Labrador_retriever,0.320012,True,Ibizan_hound,0.208172,True,Saluki,0.078975,True
1449,776201521193218049,https://pbs.twimg.com/media/CsWfKadWEAAtmlS.jpg,1,Rottweiler,0.502228,True,black-and-tan_coonhound,0.154594,True,bloodhound,0.135176,True
751,688064179421470721,https://pbs.twimg.com/media/CYx-tGaUoAAEXV8.jpg,1,Eskimo_dog,0.240602,True,Norwegian_elkhound,0.180369,True,Siberian_husky,0.090739,True
1129,728015554473250816,https://pbs.twimg.com/media/ChpuRyvVAAARMoq.jpg,1,cocker_spaniel,0.384559,True,golden_retriever,0.091661,True,sandbar,0.081799,False


In [471]:
df_image_clean['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

### code

First I am going to change the `tweet_id` from int64 into `string format`.

In [472]:
df_image_clean['tweet_id'] = df_image_clean['tweet_id'].astype('str')

In [473]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [474]:
cols = list(df_image_clean)
cols

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [484]:
df_image_clean = df_image_clean.rename(index=str, columns={"p1": "prediction_1", "p2": "prediction_2",
                                           'p3': "prediction_3", "p1_conf": "p1_confidence",
                                           "p2_conf": "p2_confidence", "p3_conf": "p3_confidence"});

### Test

In [485]:
list(df_image_clean)

['tweet_id',
 'jpg_url',
 'img_num',
 'prediction_1',
 'p1_confidence',
 'p1_dog',
 'prediction_2',
 'p2_confidence',
 'p2_dog',
 'prediction_3',
 'p3_confidence',
 'p3_dog']

## Issues `Q-6`

**some of the prediction lables are not dog! like llama, tub, web-site, etc. **

### Define

I am going to find a `complete list of dog breeds` and compare the results of the predictions and if they are not in the breed list, I am going to remove the entire record.

I found a [website](http://www.purina.com.au/owning-a-dog/dog-breed-list) with a full list of dog breeds. So I am going to `scrape the website` and get a list of all dog breeds.

will be working with `df_image_clean`.

### code

In [454]:
breed_list=[]
with open('A-Z Dog Breeds List - Purina Australia.html') as file:
    soup = BeautifulSoup(file, 'lxml')
    a = soup.find_all("div", class_="breedList")
    
    for breed in a:
        for item in breed.find_all("li"):
            breed_list.append(item.get_text(strip=True))

In [455]:
breed_df = pd.DataFrame(breed_list, columns=["breedname"])

In [515]:
# showing a sample data
breed_list[15:30]

['Beagle',
 'Bearded Collie',
 'Bedlington Terrier',
 'Belgian Shepherd Groenendael',
 'Belgian Shepherd Dog Laekenois',
 'Belgian Shepherd Dog Malinois',
 'Belgian Shepherd Tervueren',
 'Bernese Mountain Dog',
 'Bichon Frise',
 'Bloodhound',
 'Border Collie',
 'Border Terrier',
 'Borzoi',
 'Boston Terrier',
 'Bouvier des Flandres']

In [458]:
breed_df.head()

,breedname
0,Affenpinscher
1,Afghan Hound
2,Airedale Terrier
3,Alaskan Malamute
4,American Cocker Spaniel


In [516]:
df_image_clean.head()

,tweet_id,jpg_url,img_num,prediction_1,p1_confidence,p1_dog,prediction_2,p2_confidence,p2_dog,prediction_3,p3_confidence,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [497]:
sum(df_image_clean['p1_dog'] | df_image_clean['p2_dog'] | df_image_clean['p3_dog'] )

1751

Let us find those that are `NOT DOG`, according to the AI predictions (p1-p3).

In [502]:
df_image_clean[~(df_image_clean['p1_dog'] | df_image_clean['p2_dog'] | df_image_clean['p3_dog'])];

In [503]:
len(df_image_clean[~(df_image_clean['p1_dog'] | df_image_clean['p2_dog'] | df_image_clean['p3_dog'])])

324

According to the list above, we have `324 records` of `non-dog` images, so I am going to remove them all from the Data Frame.

In [504]:
df_image_clean_1 = df_image_clean[(df_image_clean['p1_dog'] | df_image_clean['p2_dog'] | df_image_clean['p3_dog'])]

In [505]:
df_image_clean_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1751 entries, 0 to 2073
Data columns (total 12 columns):
tweet_id         1751 non-null object
jpg_url          1751 non-null object
img_num          1751 non-null int64
prediction_1     1751 non-null object
p1_confidence    1751 non-null float64
p1_dog           1751 non-null bool
prediction_2     1751 non-null object
p2_confidence    1751 non-null float64
p2_dog           1751 non-null bool
prediction_3     1751 non-null object
p3_confidence    1751 non-null float64
p3_dog           1751 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 141.9+ KB


Storing the new DataFrame into a CSV file called `prediction_cleaned.csv`.

In [517]:
df_image_clean_1.to_csv('prediction_cleaned.csv')

Storing the new DataFrame into a table in a SQLite Database called `prediction.db`.

In [518]:
from sqlalchemy import create_engine

In [519]:
# Create SQLAlchemy Engine and empty prediction database
# prediction.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///prediction.db')

In [520]:
df_image_clean_1.to_sql('master', engine, index=False)

Storing the `tweet_cleaned` DataFrame into a table in a SQLite Database called `tweets.db`.

In [523]:
# Create SQLAlchemy Engine and empty prediction database
# tweets.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///tweets.db')

In [524]:
tweet_cleaned.to_sql('master', engine, index=False)

# Analysing data